## Goal:
- predict total sales for every product and store in the next month

### Data Info
- item_cnt_day: num of products sold (predicting a monthly amount of this measure)
- date_block_num: January 2013 is 0, February 2013 is 1,..., October 2015 is 33

### Procedure:
- Get the items_id from test dataset
    - Get the test items not appear in train dataset, fill with 0 vlaues for validation
    - Get the tuple(ShopID, item) for training dataset (take shop into account)
- Based on item_lst, Feature Engineering on dataset
- Apply one xgboost model 
- Apply one time series modeling

## Preprocess
- datetime
- sum up sales on a monthly basis
- Remove outliers 
- fill the month num without item_cnt_day sum with 0 value
- Fill the nan value

### Features
- seasonality
    - month
    - date_cat_avg_item_cnt
    - date_block_num
    - item_cnt_month 
- shops
    - date_shop_cat_avg_item_cnt
- items
    - item_category_id	
    - delta_price_lag
    - item_id
    - date_cat_avg_item_cnt
    - date_item_avg_item_cnt
    - item_avg_item_price
    - date_item_avg_item_price
    - price_change_percent


### Top features
- date_item_avg_item_cnt
- item_category_id	
- month
- date_cat_avg_item_cnt
- delta_price_lag
- item_id

### Label
- col `item_cnt_day`


### Model (XGBoost)
- Train data -- week 0-32
- Valid data -- week 33

### Post Process Approach
- Take 25% high (shop,item) tuple with item_cnt_month, (median of last 4 or 5 month in last year), replace the pred value with the median and clip (0,20) 
- take the median value of all shop and items from first to last year, get all (items, shop) for median with 0, replace their value with 0 &
- take the tuple not appear in train set but in test set, replace the pred val with 0


## Challenges
- matrix are too large to process
    - there are 3570291 input entries for model input
- there are huge infrequent items sold for training period
    - 75% of data have sold items less than 5 months out of 2 year and half
- (shop, items) appear in test set not in train set

### Approach
- Use spark dataframe to process large dataset in parallel
- Add more features for model training
- Train frequent and infrequent items separately
- Make the (shop, items) tuple in test set label as 0.

## Test set
- test set have 42 unique shops id, 5100 unique items, ttl shape: 214200 (5100*42)


In [1]:
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline
from xgboost import XGBRegressor
from xgboost import plot_importance
from itertools import product
import time
import pickle
from tqdm import tqdm
import os
import pandas as pd

In [2]:
# merge train with item data
def file_loc(path):
    filename = 'data/'
    file_dir = os.path.join(filename,path)
    return file_dir

train_file = file_loc('sales_train.csv')
item_file = file_loc('items.csv')
shops_file = file_loc('shops.csv')
item_cate_file = file_loc('item_categories.csv')
test = file_loc('test.csv')

df_train = pd.read_csv(train_file)
df_item = pd.read_csv(item_file)
df_shops = pd.read_csv(shops_file)
df_item_cate = pd.read_csv(item_cate_file)
df_test = pd.read_csv(test)

print('training data shape: ', df_train.shape)
print(df_train.head())

training data shape:  (2935849, 6)
         date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0  02.01.2013               0       59    22154      999.00           1.0
1  03.01.2013               0       25     2552      899.00           1.0
2  05.01.2013               0       25     2552      899.00          -1.0
3  06.01.2013               0       25     2554     1709.05           1.0
4  15.01.2013               0       25     2555     1099.00           1.0


In [3]:
matrix_1 = pd.read_csv('preprocess_matrix.csv', low_memory=False)
matrix_1.head()

,date_block_num,shop_id,item_id,date,item_price,item_cnt_day,item_name,item_category_id,month,shop_item_sum_month,...,date_cat_avg_item_cnt,item_avg_item_price,date_item_avg_item_price,year,shop_item_month_year,item_month_year,shop_day_month,item_day_month,shop_item_month,shop_item_month_un
0,0,2,32,NaN,0.0,0.0,0,0.0,0.0,0.0,...,10.812327,170.302401,336.575462,2013.0,1.0,13.0,65477,728,1007,65477
1,0,2,33,2013-01-05,499.0,1.0,1+1 (BD),37.0,1.0,1.0,...,74.580922,108.304828,391.372549,2013.0,4.0,13.0,1325,91,1007,1325
2,0,2,42,NaN,0.0,0.0,0,0.0,0.0,0.0,...,10.812327,19.731225,15.090909,2013.0,1.0,12.0,65477,1175,1007,65477
3,0,2,45,NaN,0.0,0.0,0,0.0,0.0,0.0,...,10.812327,29.558935,61.558824,2013.0,1.0,13.0,65477,1185,1007,65477
4,0,2,51,NaN,0.0,0.0,0,0.0,0.0,0.0,...,10.812327,34.104661,60.567568,2013.0,1.0,13.0,65477,1240,1007,65477


## Preprocess
- datetime
- sum up on a monthly basis
- take care of outliers -- clip
- fill the month num without item_cnt_day sum with 0 value

### Label
- col `item_cnt_day`

In [ ]:
# items in test set to train set
test_id_lst = df_test['item_id'].unique().tolist()
df_train_item = df_train_pro.loc[df_train_pro['item_id'].isin(test_id_lst)]
val = (5100-4737)/5100

print('total_item_test_len: ', len(test_id_lst))
print('total_item_train_len: ', len(df_train_item['item_id'].unique()))
print('null test_id percentage in train_data: ', val)

In [6]:
# items in test set to train set
test_item_lst = df_test['item_id'].unique().tolist()
test_shop_lst = df_test['shop_id'].unique().tolist()
df_train_item = df_train_pro.loc[(df_train_pro['item_id'].isin(test_item_lst)) & (df_train_pro['shop_id'].isin(test_shop_lst))]

In [ ]:
# process into datetime
df_train_item['date'] = pd.to_datetime(df_train_item['date'], format='%d.%m.%Y')
df_train_item['month'] = df_train_item['date'].dt.month

### Remove outliers

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(10,4))
plt.xlim(-100, 3000)
sns.boxplot(x=df_train_item['item_cnt_day'])
print('Sale volume outliers:',df_train_item['item_id'][df_train_item['item_cnt_day']>500].unique())

plt.figure(figsize=(10,4))
plt.xlim(df_train_item['item_price'].min(), df_train_item['item_price'].max())
sns.boxplot(x=df_train_item['item_price'])
print('Item price outliers:',df_train_item['item_id'][df_train_item['item_price']>50000].unique())

In [ ]:
df_train_no_out = df_train_item.loc[(df_train_item['item_cnt_day']<500) & (df_train_item['item_price']<40000)]
print('before remove outlier shape: ', df_train_item.shape)
print('after remove outlier shape: ', df_train_no_out.shape)

### Product function
- take the month, shop, and items product together

### create df
    - See all shops which sell items, items being sold at that month
    - have the rows prepared for that month with the combination of items and shops
    

In [ ]:
ts = time.time()
matrix = []
cols = ['date_block_num','shop_id','item_id']
for i in range(34):
    sales = df_train_no_out[df_train_no_out.date_block_num==i]
    matrix.append(np.array(list(product([i], sales.shop_id.unique(), sales.item_id.unique())), dtype='int16'))
    
matrix = pd.DataFrame(np.vstack(matrix), columns=cols)
matrix.sort_values(cols,inplace=True)
time.time() - ts

In [ ]:
ts = time.time()

matrix = pd.merge(matrix, df_train_no_out, on=cols, how='left')
matrix['shop_item_sum_month'] = matrix.groupby(['date_block_num','shop_id','item_id'])['item_cnt_day'].transform('sum')
matrix['shop_item_sum_month'] = (matrix['shop_item_sum_month']
                                .fillna(0)
                                .clip(0,20) # NB clip target here
                                .astype(np.float16))
time.time() - ts

### Test set Preparation

In [ ]:
df_test['date_block_num'] = 34
matrix = pd.concat([matrix, df_test], ignore_index=True, sort=False, keys=cols)
matrix.fillna(0, inplace=True) # 34 month
matrix.head()

### Fill the nan value
- item_price

In [ ]:
df_item_mean = matrix.groupby(['item_id'])['item_price'].mean().reset_index()
df_item_cate_median = matrix.groupby(['item_id'])['item_category_id'].median().reset_index()

for item in matrix['item_id'].unique().tolist():
    mean_price = df_item_mean.loc[df_item_mean['item_id']==item, 'item_price'].values[0]
    matrix.loc[(matrix['item_id']==item) & (matrix['item_price'].isnull()), 'item_price'] = mean_price
    matrix['item_cnt_day'].fillna(0, inplace=True)
    item_cate = df_item_cate_median.loc[df_item_cate_median['item_id']==item, 'item_category_id'].values[0]
    matrix.loc[(matrix['item_id']==item) & (matrix['item_category_id'].isnull()), 'item_category_id'] = item_cate
    

In [ ]:
# incorporate shop and item feature

# incorporate item feature
matrix['item_cnt_month'] = matrix.groupby(['date_block_num', 'item_id'])['item_cnt_day'].transform('sum')
# every month, items sold mean
matrix['date_item_avg_item_cnt'] = matrix.groupby(['date_block_num', 'item_id'])['item_cnt_day'].transform('mean')
# incorporate shop and cate feature
matrix['date_shop_cat_avg_item_cnt'] = matrix.groupby(['date_block_num', 'shop_id', 'item_category_id'])['item_cnt_month'].transform('mean')
# cate
matrix['date_cat_avg_item_cnt'] = matrix.groupby(['date_block_num', 'item_category_id'])['item_cnt_month'].transform('mean')
# feature delta_price
matrix['item_avg_item_price'] = matrix.groupby(['item_id'])['item_price'].transform('mean')
# 
matrix['date_item_avg_item_price'] = matrix.groupby(['date_block_num', 'item_id'])['item_price'].transform('mean')

In [ ]:
matrix['date'] = pd.to_datetime(matrix['date'], format='%Y-%m-%d', errors='coerce')
matrix['year'] = matrix['date'].dt.year
date_year_dict = {}
for month in matrix['date_block_num'].unique():
    try:
        val = matrix.loc[(matrix['date_block_num']==month) & (matrix['year'].notnull()), 'year'].values[0]
        date_year_dict[month] = val
    except:
        pass

date_year_dict[34]=2015.0
matrix['year'] = matrix['date_block_num'].map(date_year_dict)

# a year feature --  See shop sold items on nums of months
matrix['shop_item_month_year'] = matrix.groupby(['shop_id', 'item_id', 'year'])['month'].transform('nunique')
# a year feature --  See items sold on nums of months
matrix['item_month_year'] = matrix.groupby(['item_id', 'year'])['month'].transform('nunique')


# a month feature --  See shop sold on nums of days
matrix['shop_day_month'] = matrix.groupby(['shop_id','month'])['item_cnt_day'].transform('count')
# a month feature --  See items sold on nums of days
matrix['item_day_month'] = matrix.groupby(['item_id', 'month'])['item_cnt_day'].transform('count')

# a month feature
matrix['shop_item_month'] = matrix.groupby(['shop_id', 'date_block_num'])['item_id'].transform('count')
# a month feature
matrix['shop_item_month_un'] = matrix.groupby(['shop_id', 'month'])['item_id'].transform('count')
# Add 1 feature with existing sold month in a year
matrix['date_shop_cat_avg_item_cnt'] = matrix.groupby(['date_block_num', 'shop_id', 'item_category_id'])['item_cnt_month'].transform('mean')


In [ ]:
matrix_train_item_month = matrix.drop_duplicates(['date_block_num', 'item_id', 'shop_id'])
matrix_train_item_month.sort_values(by=['date_block_num'], inplace=True)
# item_price shift
matrix_train_item_month['item_price_last'] = matrix_train_item_month.groupby(['item_id'])['item_price'].shift(periods=1, fill_value=0)

### Features for model Input
- date_block_num
- item_id
- item_price (scale)
- month
- item_category_id
- item_cnt_month (scale)
- date_item_avg_item_cnt (scale)
- date_shop_cat_avg_item_cnt (scale)
- date_cat_avg_item_cnt (scale)
- item_avg_item_price (scale)
- date_item_avg_item_price (scale)
- price_change_percent(scale)
- shop_item_sum_month (label)

### Model (XGBoost)
- Train data -- week 32
- Valid data -- week 33

In [ ]:
# modify the index order
# exclude 2 cols : 'shop_item_month_year', 'item_month_year'
col_lst = ['date_block_num', 'item_id', 'shop_id', 'item_price', 'month', 'item_category_id', 'item_cnt_month', 
          'date_item_avg_item_cnt', 'date_shop_cat_avg_item_cnt', 'date_cat_avg_item_cnt', 'item_avg_item_price',
           'date_item_avg_item_price', 
         'shop_day_month', 'item_day_month', 'shop_item_month_un', 'shop_item_month']
df_model = matrix_train_item_month[col_lst]
matrix_train_item_month['shop_item_sum_month'] = matrix_train_item_month['shop_item_sum_month'].clip(0,20)
matrix_train_item_month['shop_item_sum_month'] = matrix_train_item_month['shop_item_sum_month'].astype(int)
X_train = df_model[df_model.date_block_num < 33]
X_train.sort_values(by=['date_block_num', 'shop_id','item_id'], inplace=True)
y_train = matrix_train_item_month.loc[matrix_train_item_month.date_block_num < 33, 'shop_item_sum_month'].to_frame()
# modify part
X_train_idx = X_train.index.values
y_train = y_train.reindex(X_train_idx)
Y_train = y_train.values
X_valid = df_model[df_model.date_block_num == 33]
X_valid.sort_values(by=['date_block_num', 'shop_id','item_id'], inplace=True)
y_valid = matrix_train_item_month.loc[matrix_train_item_month.date_block_num == 33, 'shop_item_sum_month'].to_frame()
# modify part
X_valid_idx = X_valid.index.values
y_valid = y_valid.reindex(X_valid_idx)
Y_valid = y_valid.values

In [ ]:
# run
model = XGBRegressor(
    max_depth=8,
    n_estimators=1000,
    min_child_weight=300, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    learning_rate=0.03,    
    seed=42)

model.fit(
    X_train, 
    Y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, Y_train), (X_valid, Y_valid)], 
    verbose=True, 
    early_stopping_rounds = 15)

### Prediction

In [ ]:
X_test = df_model[df_model.date_block_num == 34]
X_test.sort_values(by=['date_block_num', 'shop_id', 'item_id'], inplace=True)

col_lst = ['date_block_num', 'item_id', 'shop_id', 'item_price', 'month', 'item_category_id', 'item_cnt_month', 
          'date_item_avg_item_cnt', 'date_shop_cat_avg_item_cnt', 'date_cat_avg_item_cnt', 'item_avg_item_price',
           'date_item_avg_item_price', 
         'shop_day_month', 'item_day_month', 'shop_item_month_un', 'shop_item_month']
test_merge_df = X_test[col_lst]

Y_pred = model.predict(X_valid).clip(0, 20)
Y_test = model.predict(test_merge_df).clip(0, 20)

# save predictions for an ensemble
pickle.dump(Y_pred, open('xgb_train.pickle', 'wb'))
pickle.dump(Y_test, open('xgb_test.pickle', 'wb'))

### Approach
- Take 25% high (shop,item) tuple with item_cnt_month, (median of last 4 or 5 month in last year), replace the pred value with the median and clip (0,20)
- take the median value of all shop and items from first to last year, get all (items, shop) for median with 0, replace their value with 0 &
- take the tuple not appear in train set but in test set, replace the pred val with 0

In [ ]:
matrix_train_item_month = pd.read_csv('/kaggle/input/largematrix/large_matrix_trained_mdl.csv')
# load xgb model prediction
submission = pd.DataFrame({
    "ID": df_test.ID, 
    "item_cnt_month": xgb_model_pred
})

test_shop_lst = df_test['shop_id'].unique().tolist()
test_item_lst = df_test['item_id'].unique().tolist()
matrix_test_year = matrix_train_item_month.loc[(matrix_train_item_month['year']==matrix_train_item_month['year'].max()) &
                            (matrix_train_item_month['shop_id'].isin(test_shop_lst)) &
                           (matrix_train_item_month['item_id'].isin(test_item_lst))]
row, col = matrix_test_year.shape
top_30 = row*0.25
int_top_30 = "%.0f" % top_30

# extracting top 25%
large30_pct = matrix_test_year.nlargest(int(int_top_30), "shop_item_sum_month") 
top30_shop_item = list(zip(large30_pct['shop_id'], large30_pct['item_id']))

top30_shop_lst = []
df_median = matrix_test_year.groupby(['shop_id', 'item_id'])['shop_item_sum_month'].median().reset_index()
for (shop, item) in top30_shop_item:
    val = df_median.loc[(df_median['shop_id']==shop) & (df_median['item_id']==item), 'shop_item_sum_month'].values[0]
    top30_shop_lst.append(val)

submission1 = pd.merge(df_test, submission, on=['ID'])
for (shop, item), month_sales in zip(top30_shop_item, top30_shop_lst):
    
    submission1.loc[(submission1['shop_id']==shop) & (submission1['item_id']==item), 'item_cnt_month'] = month_sales
top30_shop_lst

In [ ]:
# loc train data median with 0
matrix_test_year = matrix_train_item_month.loc[
                            (matrix_train_item_month['shop_id'].isin(test_shop_lst)) &
                           (matrix_train_item_month['item_id'].isin(test_item_lst))]

df_median = matrix_test_year.groupby(['shop_id', 'item_id'])['shop_item_sum_month'].median().reset_index()
df_median0 = df_median.loc[df_median['shop_item_sum_month']==0]
med0_shop_item = list(zip(df_median0['shop_id'], df_median0['item_id']))
for (shop, item) in med0_shop_item:
    
    submission1.loc[(submission1['shop_id']==shop) & (submission1['item_id']==item), 'item_cnt_month'] = 0

df_median0.head()

In [ ]:
# loc test tuple (shop, item) only in test not in train
df_train_no_test = pd.merge(df_train, df_test, on=['shop_id', 'item_id'], how='right')
df_train_no_test1 = df_train_no_test.loc[df_train_no_test['item_cnt_day'].isnull()]
train_no_test_item_lst = df_train_no_test1['item_id'].unique().tolist()
train_no_test_shop_lst = df_train_no_test1['shop_id'].unique().tolist()

tup_lst = list(zip(df_train_no_test1['shop_id'], df_train_no_test1['item_id']))

for (shop, item) in tup_lst:
    
    submission1.loc[(submission1['shop_id']==shop) & (submission1['item_id']==item), 'item_cnt_month'] = 0

print('completed')

In [ ]:
submission2 = submission1[['ID', 'item_cnt_month']]
submission2.to_csv('xgb_submission_36.csv', index=False)